In [3]:
# Установка необходимой библиотеки
!pip install chembl_webresource_client
!pip install rdkit


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.9/34.9 MB 52.7 MB/s eta 0:00:00


In [1]:

from chembl_webresource_client.new_client import new_client
import pandas as pd
from rdkit import Chem
from rdkit.Chem import Descriptors, Lipinski


### 1. Загрузить данные и сразу по активности

In [ ]:

# https://hub.2i2c.mybinder.org/user/chembl-chembl_webresource_client-2sd6tiv1/notebooks/demo_wrc.ipynb
target = new_client.target
activity = new_client.activity

# Загрузка данных для мишени CHEMBL230 и фильтруем по активности
res = activity.filter(target_chembl_id='CHEMBL230').filter(standard_type="IC50")
# оно загружается как словарь, делаем из него dataframe
# я пытался запускать это локально но оно очень долго работает лучше запускать в colabe
# но даже в нём оно может думать оч долго
df = pd.DataFrame.from_dict(res)

print(f"кол-во записей: {len(df)}")
df.head(3)

In [2]:
# 4 MB ИНФОРМАЦИИ
df.to_csv("urablyatskachalos.csv")

NameError: name 'df' is not defined

### DROPNA

In [4]:
df_filtered = pd.read_csv("urablyatskachalos.csv")
print(f"кол-во записей до: {len(df_filtered)}")
df_filtered = df_filtered[['molecule_chembl_id', 'canonical_smiles', 'standard_relation', 'standard_value', 'standard_units']]

# Удаление записей без standard_value или SMILES
df_filtered = df_filtered.dropna(subset=['standard_value', 'canonical_smiles'])

print(f"кол-во записей по: {len(df_filtered)}")

кол-во записей до: 7979
кол-во записей по: 6977


### 3. Привести значения активности к единому формату

In [ ]:
# Оставляем только записи с единицами nM и uM

allowed_units = ['nM', 'uM']
df_filtered = df_filtered[df_filtered['standard_units'].isin(allowed_units)]

print(f"кол-во записей: {len(df_filtered)}")



df_filtered['standard_value'] = pd.to_numeric(df_filtered['standard_value'])

# Функция для конвертации в нМ
def convert_to_nM(row):
    if row['standard_units'] == 'uM':
        return row['standard_value'] * 1000
    return row['standard_value'] # Теперь здесь могут быть только 'nM'

# Применяем конвертацию

df_final = df_filtered.copy()
df_final['standard_value_nM'] = df_final.apply(convert_to_nM, axis=1)


кол-во записей: 6950

Конвертация успешно завершена. Все значения приведены к nM.


### 5. Проверить корректность молекул

In [6]:
# Функция для валидации SMILES
def validate_smiles(smiles):
    try:
        mol = Chem.MolFromSmiles(smiles)
        if mol is None:
            return False
        return True
    except:
        return False


mask = df_final['canonical_smiles'].apply(validate_smiles)
df_validated = df_final[mask]

print(f"колво валидных молекул: {len(df_validated)}")

# Финал
final_dataset = df_validated[['molecule_chembl_id', 'canonical_smiles', 'standard_value_nM']].reset_index(drop=True)
final_dataset.rename(columns={'standard_value_nM': 'IC50_nM'}, inplace=True)


final_dataset.head()

колво валидных молекул: 6950


,molecule_chembl_id,canonical_smiles,IC50_nM
0,CHEMBL297008,Cc1ccc(-c2ccc(S(C)(=O)=O)cc2)n1-c1ccccc1,60.00
1,CHEMBL289813,Cc1c(C=O)cc(-c2ccc(S(C)(=O)=O)cc2)n1-c1ccc(F)cc1,3230.00
2,CHEMBL43736,Cc1c(COc2cccc(Cl)c2)cc(-c2ccc(S(C)(=O)=O)cc2)n...,80.00
3,CHEMBL140167,Fc1ccc(-c2[nH]c(-c3ccc(F)cc3)c3c2C2CCC3CC2)cc1,0.12
4,CHEMBL44194,CCc1ccc(-c2ccc(S(C)(=O)=O)cc2)n1-c1ccc(F)cc1,100000.00


### 4. Обработать пропущенные значения и дубликаты

In [ ]:
# Удаление дубликатов по SMILES

df_final = df_final.sort_values('standard_value_nM', ascending=False)
df_final = df_final.drop_duplicates('canonical_smiles', keep='first')
df_final.to_csv("prep.csv")

print(f"кол-во записей: {len(df_final)}")


кол-во записей: 5100
